In [102]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

data1 = pd.read_csv('/content/dadosconcatenadosarea1.csv', header=(0))

#Remover colunas
data1.drop(columns=['Aluno', 'Bolsista', 'Pos_Doc',
       'Aluno_Externo', 'Funcionario', 'Visitantes', 'Comunidade', 'sobremesa_2'], inplace=True)

#Remover Nan
data1 = data1.dropna()
data1 = data1.drop_duplicates()

data2 = pd.read_csv('/content/dadosconcatenadosarea2.csv', header=(0))

#Remover colunas
data2.drop(columns=['Aluno', 'Bolsista', 'Pos_Doc',
       'Aluno_Externo', 'Funcionario', 'Visitante', 'Comunidade', 'sobremesa_2','refeicao'], inplace=True)

#Remover Nan
data2 = data2.dropna()
data2 = data2.drop_duplicates()

In [103]:
data1.head()

,Ano,Mês,Dia_semana,Dia_mês,refeicao,prato_principal_1,prato_principal_2,guarnição,sobremesa_1,Total,Precipitacao_mm,Temp_max_C,Temp_min_C,Umid_rel_ar,Vento_velocidade_horaria (m/s),Vento_rajada_maxima (m/s),dia letivo
1,2023,1,ter,3,almoco,frango empanado,quibe de brócolis,creme de milho,doce de leite,207.0,0.0,25.233333,23.266667,72.000000,2.066667,7.166667,0
2,2023,1,ter,3,jantar,linguiça acebolada,ovo frito,farofa,maxi goiabinha,183.0,0.0,23.100000,22.133333,84.333333,1.166667,3.633333,0
4,2023,1,qua,4,almoco,bisteca ao molho barbecue,hamburguer de lentilha a pizzaiolo,couve refogada,barra de cereal,247.0,0.0,22.300000,21.066667,84.666667,3.366667,8.366667,0
5,2023,1,qua,4,jantar,lagarto ao molho madeira,goulash vegetariano,polenta,gelatina de morango,181.0,3.8,21.666667,19.866667,93.666667,2.166667,9.066667,0
7,2023,1,qui,5,almoco,peixe frito,omelete de forno,purê de mandioquinha,chocolate,281.0,1.6,19.566667,18.966667,94.333333,0.166667,2.900000,0


In [104]:
# --- AGRUPAR CARDAPIO ---

def agrupar_nomes(data, xlsx, sheet_name, column_in_data):
  data_agrupamento = pd.read_excel(xlsx, sheet_name, header=(0))

  # Transpor para ficar mais fácil de tratar
  data_agrupamento_transposed = data_agrupamento.T

  #Itere pela coluna, pegue todos os valores dela
  grupos = []

  for name, col in data_agrupamento_transposed.items():
    #Remove os nans
    categoria_variacoes = list(filter(lambda x: not pd.isna(x), col.values))
    grupos.append(categoria_variacoes)


  # -----------------------
  # Substitui nomes

  # Criar dicionário
  substituicoes = {}

  #Aplicando a substituição
  for array_categoria in grupos:
    substituicoes[array_categoria[0]] = array_categoria


  for prato_padrao, variacoes in substituicoes.items():
      data[column_in_data] = data[column_in_data].replace(variacoes, prato_padrao)

  return data

#--- AGRUPAR CARDAPIO ---

data1['prato_principal_1'] = data1['prato_principal_1'].str.strip()
data1['prato_principal_2'] = data1['prato_principal_2'].str.strip()
data1['guarnição'] = data1['guarnição'].str.strip()
data1['sobremesa_1'] = data1['sobremesa_1'].str.strip()


#Importar os dados

xlsx = pd.ExcelFile('/content/agrupamento_de_nomes.xlsx')

data1 = agrupar_nomes(data1, xlsx, 'Carnes', 'prato_principal_1')
data1 = agrupar_nomes(data1, xlsx, 'Vegetariano', 'prato_principal_2')
data1 = agrupar_nomes(data1, xlsx, 'Guarnição', 'guarnição')
data1 = agrupar_nomes(data1, xlsx, 'Sobremesa doce', 'sobremesa_1')

#--- AGRUPAR CARDAPIO ---

data2['prato_principal_1'] = data2['prato_principal_1'].str.strip()
data2['prato_principal_2'] = data2['prato_principal_2'].str.strip()
data2['guarnição'] = data2['guarnição'].str.strip()
data2['sobremesa_1'] = data2['sobremesa_1'].str.strip()


#Importar os dados

xlsx = pd.ExcelFile('/content/agrupamento_de_nomes.xlsx')

data2 = agrupar_nomes(data2, xlsx, 'Carnes', 'prato_principal_1')
data2 = agrupar_nomes(data2, xlsx, 'Vegetariano', 'prato_principal_2')
data2 = agrupar_nomes(data2, xlsx, 'Guarnição', 'guarnição')
data2 = agrupar_nomes(data2, xlsx, 'Sobremesa doce', 'sobremesa_1')

ÁREA1 - limpando observações estranhas

In [105]:
data1 = data1[data1["prato_principal_1"] != "File de peixe ou frango"]

In [106]:
data1['prato_principal_1'].value_counts()

,count
prato_principal_1,
Bife,206
Lombo,115
frango assado,108
Frango empanado,91
Pernil,70
Carne em cubos,66
Filé de frango,61
Carne moída,60
Linguiça,60


In [107]:
data1 = data1[data1["prato_principal_2"] != "não informado"]

In [108]:
data1['prato_principal_2'].value_counts()

,count
prato_principal_2,
PVT,154
Omelete,104
Ovo frito,90
Curry de legumes,66
Quibe de brócolis,65
Almôndega de aveia,61
Tomate recheado,53
Suflê de legumes,50
Moussaka,48


In [109]:
data1 = data1[data1["guarnição"] != "não informado"]

In [110]:
data1['guarnição'].value_counts()

,count
guarnição,
Farofa,111
Macarrão,96
Polenta,95
Legumes,91
Batata,85
Batata palha,77
Chuchu,69
Creme de milho,58
Ervilha parmentier,55


In [111]:
data1['sobremesa_1'].value_counts()

,count
sobremesa_1,
Gelatina,93
Bolinho,89
Barra de cereais,87
Goiabada,78
Chocolate,69
Bombom,60
Picolé,59
Iogurte,55
Brigadeirão,49


ÁREA2

In [112]:
data2 = data2[data2["prato_principal_1"] != "File de peixe ou frango"]

In [113]:
data2['prato_principal_1'].value_counts()

,count
prato_principal_1,
Bife,73
Frango empanado,52
Lombo,39
frango assado,38
Carne moída,38
Pernil,31
Linguiça,27
Carne em cubos,27
Lasanha bolonhesa,25


In [114]:
data2 = data2[data2["prato_principal_2"] != "não informado"]

In [115]:
data2['prato_principal_2'].value_counts()

,count
prato_principal_2,
PVT,60
Ovo frito,35
Omelete,35
Quibe de brócolis,25
Curry de legumes,25
Lasanha de PVT,22
Almôndega de aveia,21
Tomate recheado,20
Suflê de legumes,20


In [116]:
data2 = data2[data2["guarnição"] != "não informado"]

In [117]:
data2['guarnição'].value_counts()

,count
guarnição,
Macarrão,45
Legumes,43
Polenta,36
Farofa,35
Creme de milho,32
Batata palha,32
Batata,29
Couve,26
Purês,23


In [118]:
data2['sobremesa_1'].value_counts()

,count
sobremesa_1,
Gelatina,43
Barra de cereais,41
Bolinho,34
Goiabada,30
Chocolate,27
Picolé,22
Brigadeirão,21
Manjar,20
Bombom,20


In [119]:
from google.colab import files

data1.to_csv("data_area1.csv", index=False)
files.download("data_area1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [120]:
from google.colab import files

data2.to_csv("data_area2.csv", index=False)
files.download("data_area2.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>